In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [18]:
## Section 2: Dataset Preparation
data_dir = 'Gemstones'  # Ensure this directory exists and contains subdirectories for each class

# Validate Dataset
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Dataset directory {data_dir} not found.")

categories = os.listdir(data_dir)
categories = [cat for cat in categories if os.path.isdir(os.path.join(data_dir, cat))]
num_classes = len(categories)

print(f"Categories Found: {categories}")
print(f"Number of Classes: {num_classes}")

if num_classes == 0:
    raise ValueError("No categories found in the dataset directory.")

Categories Found: ['Alexandrite', 'Ametrine', 'Andalusite', 'Andradite', 'Aquamarine', 'Aventurine Green', 'Aventurine Yellow', 'Benitoite', 'Beryl Golden', 'Carnelian', 'Cats Eye', 'Chalcedony', 'Chalcedony Blue', 'Coral', 'Diaspore', 'Dumortierite', 'Emerald', 'Fluorite', 'Garnet Red', 'Goshenite', 'Grossular', 'Hessonite', 'Hiddenite', 'Iolite', 'Jade', 'Jasper', 'Kunzite', 'Kyanite', 'Labradorite', 'Lapis Lazuli', 'Larimar', 'Malachite', 'Moonstone', 'Morganite', 'Opal', 'Pearl', 'Peridot', 'Pyrite', 'Pyrope', 'Quartz Beer', 'Quartz Rose', 'Quartz Rutilated', 'Rhodochrosite', 'Rhodolite', 'Sapphire Blue', 'Sunstone', 'Tanzanite', 'Variscite', 'Zircon', 'Zoisite']
Number of Classes: 50


In [19]:
## Section 3: ImageDataGenerator
img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

# Debugging Data Generator
for data_batch, labels_batch in train_generator:
    print(f"Image batch shape: {data_batch.shape}")
    print(f"Labels batch shape: {labels_batch.shape}")
    break

Found 1338 images belonging to 50 classes.
Found 312 images belonging to 50 classes.
Image batch shape: (32, 224, 224, 3)
Labels batch shape: (32, 50)


In [20]:
## Section 4: Model Definition
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')  # Ensure num_classes matches categories
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [21]:
## Section 5: Model Training
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss:.2f}, Validation Accuracy: {val_acc:.2f}")

model.save('gemstones10_model.h5')
print("Model saved as 'gemstones10_model.h5'")

Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 170s 4s/step - accuracy: 0.0756 - loss: 4.6814 - val_accuracy: 0.1699 - val_loss: 3.3022
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 158s 4s/step - accuracy: 0.3984 - loss: 2.3768 - val_accuracy: 0.3365 - val_loss: 2.4794
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 159s 4s/step - accuracy: 0.6816 - loss: 1.3490 - val_accuracy: 0.3942 - val_loss: 2.2356
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 158s 4s/step - accuracy: 0.8195 - loss: 0.8099 - val_accuracy: 0.4231 - val_loss: 2.1549
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 157s 4s/step - accuracy: 0.8977 - loss: 0.5144 - val_accuracy: 0.4712 - val_loss: 2.0404
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 159s 4s/step - accuracy: 0.9649 - loss: 0.2918 - val_accuracy: 0.4615 - val_loss: 1.9649
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 160s 4s/step - accuracy: 0.9773 - loss: 0.2149 - val_accuracy: 0.4808 - val_loss: 1.9150
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 162s 4s/step - accuracy: 0.9865 - loss: 0.1490 - val_accuracy: 0.5096 - v

Validation Loss: 1.89, Validation Accuracy: 0.52
Model saved as 'gemstones10_model.h5'


In [22]:
## Section 6: Model Testing and Prediction
try:
    # Load the model in the new Keras format
    model = load_model('gemstones10_model.keras')
    print("Model loaded successfully.")
    model.summary()

    # Create a dummy input for testing
    example_input = np.random.random((1, 224, 224, 3)).astype(np.float32)

    # Make predictions
    prediction = model.predict(example_input)
    predicted_class = categories[np.argmax(prediction)]
    print(f"Predicted Class: {predicted_class}")

    # Display dummy image with prediction
    plt.imshow(example_input[0])
    plt.title(f"Predicted Class: {predicted_class}")
    plt.show()

except Exception as e:
    print(f"Error during model testing: {e}")


Error during model testing: File not found: filepath=gemstones10_model.keras. Please ensure the file is an accessible `.keras` zip file.


In [23]:
## Section 7: Fine-tuning the Model (Optional)
train_data = np.random.random((100, 224, 224, 3))  # Replace with actual training data
train_labels = np.random.randint(0, len(categories), (100,))  # Replace with actual labels

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=10)
model.save('updated_gemstones10_model.h5')
print("Updated model saved as 'updated_gemstones10_model.h5'")

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.0525 - loss: 6.2666
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.0646 - loss: 4.7576
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.0182 - loss: 4.2341 
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.0679 - loss: 3.8079
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.0770 - loss: 3.6169
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.0829 - loss: 3.6123
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.1107 - loss: 3.5298
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.1056 - loss: 3.4903
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.1932 - loss: 3.3539
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.1542 - loss: 3.3571


Updated model saved as 'updated_gemstones10_model.h5'
